# F-statistic tutorial

We will use `PyFstat`.

## Information about `PyFstat`
- "This is a python package providing an interface to perform F-statistic based continuous gravitational wave (CW) searches, built on top of the LALSuite library." (from github)
- [github](https://github.com/PyFstat/PyFstat)
- [document](https://pyfstat.readthedocs.io/en/latest/)
- Developers  
  - Maintainers: Greg Ashton, David Keitel  
  - Active contributors: Reinhard Prix, Rodrigo Tenorio  
  - Other contributors: Karl Wette, Sylvia Zhu, Dan Foreman-Mackey (pyfstat.gridcorner is based on DFM's corner.py)



## Install package

First, you install the module `pyfstat`. At this moment, `numpy` will be reinstalled (Google colab preinstalled numpy, but it is not compatible with the pyfstat's requirement). After installing `numpy` and `pyfstat`, you need to restart the notebook. You just follow the popup.

In [ ]:
!pip install pyfstat

You mount your google drive directory on this notebook. After that, you change the working directory to `2024_OpenDataWorkshop_cw`.

In [ ]:
#----------------------
# Setting up the environment
#----------------------
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/2024_OpenDataWorkshop_cw
!ls

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pyfstat
import lal
lal.swig_redirect_standard_output_error(True)
%matplotlib inline

## Search over $f^{(0)}$

In [ ]:
label = "PyFstatExampleGridSearchF0"
outdir = os.path.join("Mockdata", label)
logger = pyfstat.set_up_logger(label=label, outdir=outdir)

In [ ]:
sftfilepath = "Mockdata/H-7008_H1_1800SFT_SingleDetectorGaussianNoiseSignalInjected-1238166018-12614400.sft"
tstart = 1238166018
duration = 0.4 * 365 * 86400
detectors = "H1"
Tsft = 1800
depth = 5
sqrtSX = 1.0e-23
inj = {
    "F0": 100.0,
    "F1": -1e-9,
    "F2": 0.0,
    "Alpha": 0.0,
    "Delta": 0.0,
    "h0": sqrtSX / depth,
    "cosi": 1,
    "psi": 0.0,
    "phi": 0.0,
    "tref": tstart
}

In [ ]:
# Define grids and run the search
# It will take 3-4 minutes.
tend = tstart + duration
tref = inj['tref']
ngrid = 201     # The number of grid points
dF0 = 1.0e-8    # Grid width in F0[Hz]
DeltaF0 = (ngrid - 1) * dF0   # The range of F0 to be searched

# The followings (F0s, F1s, F2s, Alphas, and Deltas) are the arguments of the function `pyfstat.GridSearch`.
# They must be lists.
# If you provide a single value, it means you fix that parameter.
# If you provide three values, it shows the setting of grid search:
# [xmin, xmax, dx]
# xmin: minimum value of the search range
# xmax: maximum value of the search range
# dx: grid width
F0s = [inj["F0"] - DeltaF0 / 2.0, inj["F0"] + DeltaF0 / 2.0, dF0]
F1s = [inj["F1"]]
F2s = [inj["F2"]]
Alphas = [inj["Alpha"]]
Deltas = [inj["Delta"]]

# Carry our the grid search
search = pyfstat.GridSearch(
    label=label,
    outdir=outdir,
    sftfilepattern=sftfilepath,
    F0s=F0s,
    F1s=F1s,
    F2s=F2s,
    Alphas=Alphas,
    Deltas=Deltas,
    tref=tref,
    minStartTime=tstart,
    maxStartTime=tend,
)
search.run()

In [ ]:
# report details of the maximum point
max_dict = search.get_max_twoF()
logger.info(
    "max2F={:.4f} from GridSearch, offsets from injection: {:s}.".format(
        max_dict["twoF"],
        ", ".join(
            [
                "{:.4e} in {:s}".format(max_dict[key] - inj[key], key)
                for key in max_dict.keys()
                if not key == "twoF"
            ]
        ),
    )
)
search.generate_loudest()

In [ ]:
logger.info("Plotting 2F(F0)...")
fig, ax = plt.subplots()
frequencies = search.data["F0"]
twoF = search.data["twoF"]
ax.plot(frequencies - inj["F0"], twoF, "k", lw=1)
ax.set_xlabel("$f^{(0)} - f^{(0)}_\\mathrm{inj}$")
ax.set_ylabel("$\\widetilde{2\\mathcal{F}}$")
ax.set_xlim([- DeltaF0 / 2.0, DeltaF0 / 2.0])
ax.set_title(f"max[2F] = {np.max(twoF)}")
plt.tight_layout()
fig.savefig(os.path.join(outdir, label + "_1D.png"), dpi=300)

## Search over 2 dimensional parameter space $(f^{(0)}, f^{(1)})$

In [ ]:
label = "PyFstatExampleGridSearchF0F1"
outdir = os.path.join("Mockdata", label)
logger = pyfstat.set_up_logger(label=label, outdir=outdir)

In [ ]:
# Define grids and run the search
# It will take 3-4 minutes.
ngrid_f0 = 41
ngrid_f1 = 41
dF0 = 1.0e-8
DeltaF0 = (ngrid_f0 - 1) * dF0
dF1 = 1.0e-15
DeltaF1 = (ngrid_f1 - 1) * dF1
F0s = [inj["F0"] - DeltaF0 / 2.0, inj["F0"] + DeltaF0 / 2.0, dF0]
F1s = [inj["F1"] - DeltaF1 / 2.0, inj["F1"] + DeltaF1 / 2.0, dF1]
F2s = [inj["F2"]]
Alphas = [inj["Alpha"]]
Deltas = [inj["Delta"]]
search = pyfstat.GridSearch(
    label=label,
    outdir=outdir,
    sftfilepattern=sftfilepath,
    F0s=F0s,
    F1s=F1s,
    F2s=F2s,
    Alphas=Alphas,
    Deltas=Deltas,
    tref=tref,
    minStartTime=tstart,
    maxStartTime=tend,
)
search.run()

In [ ]:
logger.info("Plotting 2F(F0)...")
search.plot_1D(xkey="F0", xlabel="freq [Hz]", ylabel="$2\\mathcal{F}$")
logger.info("Plotting 2F(F1)...")
search.plot_1D(xkey="F1")
logger.info("Plotting 2F(F0,F1)...")
search.plot_2D(xkey="F0", ykey="F1", colorbar=True)

logger.info("Making gridcorner plot...")
F0_vals = np.unique(search.data["F0"]) - inj["F0"]
F1_vals = np.unique(search.data["F1"]) - inj["F1"]
twoF = search.data["twoF"].reshape((len(F0_vals), len(F1_vals)))
xyz = [F0_vals, F1_vals]
labels = [
    "$f - f_\mathrm{inj}$",
    "$\\dot{f} - \\dot{f}_\mathrm{inj}$",
    "$\\widetilde{2\\mathcal{F}}$",
]
fig, axes = pyfstat.gridcorner(
    twoF, xyz, projection="log_mean", labels=labels, whspace=0.1, factor=1.8
)
fig.savefig(os.path.join(outdir, label + "_projection_matrix.png"))

Q1. How does the contour plot of the posterior look like? What is the origin of this shape?  
Q2. How large is the size of the contour? Why the contour have such a size?